In [ ]:
conda install -c conda-forge ipyleaflet

In [ ]:
pip install haversine 

In [ ]:
from IPython import get_ipython
from ipyleaflet import Map, Marker, basemaps
import pandas as pd
import requests

In [ ]:
center = (47.016667, 28.833333)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=7)
marker = Marker(location=center, draggable=True)
m.add_layer(marker)
display(m)
def handle_click(**kwargs):
    print('Широта, долгота: ', kwargs['coordinates'])
marker.on_click(handle_click)
#Чтобы узнать координаты, переместите маркер на нужную точку и нажмите на него

In [ ]:
#Задай координаты точки в десятичном виде (lat, lon) #X = (47.016667, 28.833333) 
X1 = float(input('Задай широту в десятичном виде:\n'))
X2 = float(input('Задай долготу в десятичном виде:\n'))

In [ ]:
#Функция по определению выработки СЭС
def PV_Energy(Peak_Power, X1, X2):
    X = (X1, X2)
    Optim_an = False
    Optim_az = False
    u=()
    Optim_angles = str(input('Оптимизировать угол установки? (Да/Нет)\n'))
    if Optim_angles.lower() == "да":
        Optim_an = True
    else:
        Angle = float(input('Задай угол установки:\n'))
    Optim_azimut = str(input('Оптимизировать азимут установки? (Да/Нет)\n'))
    if Optim_azimut.lower() == "да":
        Optim_az = True
    else:
        Azimut = float(input('Задай азимут установки:\n'))
    if (Optim_an is True) and Optim_az is True: #Оптим. все
        u = 'https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat=' + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power) + '&pvtechchoice=crystSi&loss=14&optimalangles=1&outputformat=json'
    elif (Optim_an is False) and Optim_az is False: #Задай все
        u = 'https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat=' + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power) + '&pvtechchoice=crystSi&loss=14&angle=' + str(Angle) + '&aspect=' + str(Azimut) + '&outputformat=json'
    elif (Optim_an is True) and Optim_az is False: #Задай все
        u = 'https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat=' + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power) + '&pvtechchoice=crystSi&loss=14&optimalinclination=1&aspect=' + str(Azimut) + '&outputformat=json'
    elif (Optim_an is False) and Optim_az is True: #Задай все
        u = 'https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?' + 'lat=' + str(X1) + '&lon=' + str(X2) + '&peakpower=' + str(Peak_Power) + '&pvtechchoice=crystSi&loss=14&angle=' + str(Angle) + '&aspect=0&outputformat=json'
    url = u
    r = requests.get(url)
    json = r.json()
    Z = json['outputs']['totals']['fixed']['E_y']
    Ugol = json['inputs']['mounting_system']['fixed']['slope']['value']
    Azimut = json['inputs']['mounting_system']['fixed']['azimuth']['value']
    return(Z, Ugol, Azimut)

In [ ]:
#Функция по определению мощности СЭС из условия необходимой выработки
One_kW_PV = PV_Energy(1, X1, X2)
One_kW_PV_Energy = One_kW_PV[0]
PV_Ugol = One_kW_PV[1]
PV_Azimuth = One_kW_PV[2]
def PV_Power(Cons):
    Solar_Power = round(Cons/One_kW_PV_Energy, 2)
    return(Solar_Power)

In [ ]:
#Калькулятор выработки СЭС по известной мощности
print('Выработка СЭС составит = ', 
      PV_Energy(float(input('Задай мощность СЭС, кВт = ')), 
                X1, X2)[0], 'кВт*ч в год')
print('Угол установки =', PV_Ugol, 'град.\n'
     'Азимут установки =', PV_Azimuth, ' град.')

In [ ]:
#Калькулятор мощности СЭС по известному потреблению
print('Необходимая мощность СЭС = ', PV_Power(float(input('Задай потребление, кВт*ч/год = '))), 'кВт')
print('Угол установки =', PV_Ugol, 'град.\n'
     'Азимут установки =', PV_Azimuth, ' град.')